In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

# Load Data

In [15]:
## load sp500 data
path_sp500 = "../data/market_data/SP500_HistoricalPrices.csv"
df_sp500 = pd.read_csv(path_sp500, usecols=["Date", " Close"])
df_sp500['Date'] = df_sp500['Date'].astype('datetime64[ns]')
df_sp500 = df_sp500.iloc[::-1]

# calculate return of the day from closing price 
df_sp500["one_day_return"] = df_sp500[" Close"].pct_change(1)*100


## load treasury yield data  
# Sorce: https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=all
df_treasury = pd.read_csv("../data/market_data/daily-treasury-rates.csv", usecols=["Date", "1 Yr", "10 Yr", "3 Mo"])
df_treasury['Date'] = df_treasury['Date'].astype('datetime64[ns]')
df_treasury['slope_10_1'] = df_treasury['10 Yr'] - df_treasury['1 Yr']
df_treasury['slope_10y_3m'] = df_treasury['10 Yr'] - df_treasury['3 Mo']
df_treasury = df_treasury.iloc[::-1]


## load aggregate measure data for meeting minutes
df_measure = pd.read_excel("../data/market_data/aggregate_measure.xlsx", usecols=["ReleaseDate", "our_measure", "EndDate"])
df_measure["EndDate"] = pd.to_datetime(df_measure["EndDate"], format='%B/%d/%Y')
print("Mean of measure: ", df_measure["our_measure"].mean())
print("Std of measure: ", df_measure["our_measure"].std())
df_measure["delay"] = (df_measure["ReleaseDate"] - df_measure["EndDate"]).dt.days


## load aggregate measure data for speeches
df_measure_speeches = pd.read_excel("../data/market_data/aggregate_measure_speech.xlsx", usecols=["Date", "our_measure", "number_of_filtered_sent", "Speaker"])
df_measure_speeches['Date'] = df_measure_speeches['Date'].astype('datetime64[ns]')
df_measure_speeches = df_measure_speeches.loc[df_measure_speeches['number_of_filtered_sent'] > 10]

def check_chair(speaker):
    if speaker[0:5] == "Chair":
        return 1
    else:
        return 0
df_measure_speeches['Chair'] = df_measure_speeches["Speaker"].apply(lambda x: check_chair(x))
df_measure_speeches_chair = df_measure_speeches.loc[df_measure_speeches['Chair'] == 1]
print("Mean of measure (Speeches): ", df_measure_speeches["our_measure"].mean())
print("Std of measure (Speeches): ", df_measure_speeches["our_measure"].std())


## load aggregate measure data for testimonies
df_measure_testimonies = pd.read_excel("../data/market_data/aggregate_measure_testimony.xlsx", usecols=["Date", "our_measure", "number_of_filtered_sent"])
df_measure_testimonies['Date'] = df_measure_testimonies['Date'].astype('datetime64[ns]')
df_measure_testimonies = df_measure_testimonies.loc[df_measure_testimonies['number_of_filtered_sent'] > 10]
print("Mean of measure (Testimonies): ", df_measure_testimonies["our_measure"].mean())
print("Std of measure (Testimonies): ", df_measure_testimonies["our_measure"].std())


Mean of measure:  -0.07036516881815903
Std of measure:  0.25285972934531076
Mean of measure (Speeches):  -0.16540441916008608
Std of measure (Speeches):  0.2649489325956476
Mean of measure (Testimonies):  -0.1728534318526911
Std of measure (Testimonies):  0.30086316687490094


In [16]:
######################## Predict S&P returns from residual on 10Yr - 1Yr Treasury  ########################
## For Meeting Minutes
date_type = "ReleaseDate"#"EndDate" 

df_merge = pd.merge(df_measure, df_treasury, left_on=date_type, right_on="Date", how="left")

df_merge = df_merge.dropna()

# run OLS
var_x = df_merge["our_measure"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge["10 Yr"].tolist() #"1 Yr", "10 Yr", "slope_10_1", slope_10y_3m, 3 Mo

result = sm.OLS(var_y, var_x).fit()
#print(result.summary())


### get residual from regression
pred_y = result.predict(var_x)
residual_list = [(var_y[i]-pred_y[i]) for i in range(len(var_x)) ]

df_merge["slope_10_1_residual"] = pd.Series(residual_list)
df_measure_residual = df_merge[[date_type, "slope_10_1_residual"]]
df_measure_residual = df_measure_residual.dropna()


######################## one day window analysis (% points) ########################
# calculate return of the day from closing price 
df_sp500["one_day_return"] = df_sp500[" Close"].pct_change(1)*100

df_merge_sp = pd.merge(df_measure_residual, df_sp500, left_on=date_type, right_on="Date", how="left")

# run OLS
var_x = df_merge_sp["slope_10_1_residual"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge_sp["one_day_return"].tolist()

result = sm.OLS(var_y, var_x).fit()
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     6.271
Date:                Fri, 03 Jun 2022   Prob (F-statistic):             0.0131
Time:                        00:33:14   Log-Likelihood:                -304.90
No. Observations:                 198   AIC:                             613.8
Df Residuals:                     196   BIC:                             620.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0374      0.081      0.464      0.6

In [19]:
######################## Predict S&P returns from residual on 10Yr - 1Yr Treasury  ########################
## For Speeches
df_merge = pd.merge(df_measure_speeches, df_treasury, left_on="Date", right_on="Date", how="left")

df_merge = df_merge.dropna()

# run OLS
var_x = df_merge["our_measure"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge["slope_10_1"].tolist() #"1 Yr", "10 Yr", "slope_10_1", slope_10y_3m, 3 Mo

result = sm.OLS(var_y, var_x).fit()
#print(result.summary())


### get residual from regression
pred_y = result.predict(var_x)
residual_list = [(var_y[i]-pred_y[i]) for i in range(len(var_x)) ]

df_merge["slope_10_1_residual"] = pd.Series(residual_list)
df_measure_residual = df_merge[["Date", "slope_10_1_residual"]]
df_measure_residual = df_measure_residual.dropna()


######################## one day window analysis (% points) ########################
# calculate return of the day from closing price 
df_sp500["one_day_return"] = df_sp500[" Close"].pct_change(1)*100

df_merge_sp = pd.merge(df_measure_residual, df_sp500, left_on="Date", right_on="Date", how="left")

# run OLS
var_x = df_merge_sp["slope_10_1_residual"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge_sp["one_day_return"].tolist()

result = sm.OLS(var_y, var_x).fit()
print(result.summary())




                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     8.205
Date:                Fri, 03 Jun 2022   Prob (F-statistic):            0.00433
Time:                        00:35:10   Log-Likelihood:                -962.23
No. Observations:                 598   AIC:                             1928.
Df Residuals:                     596   BIC:                             1937.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0250      0.050      0.504      0.6

In [12]:
######################## Predict S&P returns from residual on 10Yr - 1Yr Treasury  ########################
## For Testimonies
df_merge = pd.merge(df_measure_testimonies, df_treasury, left_on="Date", right_on="Date", how="left")

df_merge = df_merge.dropna()

# run OLS
var_x = df_merge["our_measure"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge["slope_10_1"].tolist() #"1 Yr", "10 Yr", "slope_10_1", slope_10y_3m, 3 Mo

result = sm.OLS(var_y, var_x).fit()
#print(result.summary())


### get residual from regression
pred_y = result.predict(var_x)
residual_list = [(var_y[i]-pred_y[i]) for i in range(len(var_x)) ]

df_merge["slope_10_1_residual"] = pd.Series(residual_list)
df_measure_residual = df_merge[["Date", "slope_10_1_residual"]]
df_measure_residual = df_measure_residual.dropna()


######################## one day window analysis (% points) ########################
# calculate return of the day from closing price 
df_sp500["one_day_return"] = df_sp500[" Close"].pct_change(1)*100

df_merge_sp = pd.merge(df_measure_residual, df_sp500, left_on="Date", right_on="Date", how="left")

# run OLS
var_x = df_merge_sp["slope_10_1_residual"].tolist()
var_x = sm.add_constant(var_x)

var_y = df_merge_sp["one_day_return"].tolist()

result = sm.OLS(var_y, var_x).fit()
print(result.summary())




                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     14.74
Date:                Fri, 03 Jun 2022   Prob (F-statistic):           0.000175
Time:                        00:09:01   Log-Likelihood:                -249.17
No. Observations:                 167   AIC:                             502.3
Df Residuals:                     165   BIC:                             508.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1349      0.095     11.981      0.0